<a href="https://colab.research.google.com/github/sarathpanat/CNN-dog-breed-prediction/blob/master/CNN-resnet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
from glob import glob

dog_files = np.array(glob("/content/drive/My Drive/project-dog-classification/project-dog-classification/dogImages/*/*/*"))

print('There are %d total dog images.' % len(dog_files))

There are 8351 total dog images.


In [ ]:
import torch
import torchvision.models as models

import torch.nn as nn
# define VGG16 model
resnet18 = models.resnet18(pretrained=True)

# check if CUDA is available otherwise use cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# move model to GPU if CUDA is available or to cpu
resnet18 = resnet18.to(device)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth



cuda:0


In [ ]:
import os
from torchvision import datasets
import torchvision.transforms as transforms

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(20),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/drive/My Drive/project-dog-classification/project-dog-classification/dogImages/'

images_data = {p: datasets.ImageFolder(os.path.join(data_dir, p), 
                                       data_transforms[p]) 
               for p in ['train', 'valid', 'test']}

dataloaders = {p: torch.utils.data.DataLoader(images_data[p], batch_size=64, 
                                              shuffle=True, num_workers=4)
               for p in ['train', 'valid', 'test']}

In [ ]:
from collections import OrderedDict
for param in resnet18.parameters():
    param.requires_grad = False

fc = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(512,100)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(100,len(images_data['train'].classes))),
    ('output', nn.LogSoftmax(dim=1))
]))

resnet18.fc = fc
resnet18.to(device)

print(resnet18)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

### TODO: select optimizer
# optimizer_scratch = optim.SGD(model_scratch.parameters(), lr=0.001, momentum=0.9)
optimizer_scratch = optim.Adam(resnet18.parameters(), lr=0.001)

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, device, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU or CPU
            data, target = data.to(device), target.to(device)
            
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            
            loss.backward()
            optimizer.step()
            
            train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU or CPU
            data, target = data.to(device), target.to(device)
            
            ## update the average validation loss
            output = model(data)
            loss = criterion(output, target)
            
            valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Saving model..')
            valid_loss_min = valid_loss
            torch.save(model.state_dict(), save_path)
            
    # return trained model
    return model

In [ ]:
resnet18 = train(30, dataloaders, resnet18, optimizer_scratch, 
                      criterion_scratch, device, 'resnet18.pt')

Epoch: 1 	Training Loss: 4.062169 	Validation Loss: 2.449356
Saving model..
Epoch: 2 	Training Loss: 2.232296 	Validation Loss: 1.253476
Saving model..
Epoch: 3 	Training Loss: 1.649250 	Validation Loss: 0.902031
Saving model..
Epoch: 4 	Training Loss: 1.413378 	Validation Loss: 0.816941
Saving model..
Epoch: 5 	Training Loss: 1.289135 	Validation Loss: 0.755160
Saving model..
Epoch: 6 	Training Loss: 1.220677 	Validation Loss: 0.631908
Saving model..
Epoch: 7 	Training Loss: 1.193584 	Validation Loss: 0.638202
Epoch: 8 	Training Loss: 1.171394 	Validation Loss: 0.672154
Epoch: 9 	Training Loss: 1.107610 	Validation Loss: 0.584111
Saving model..
Epoch: 10 	Training Loss: 1.066014 	Validation Loss: 0.681923
Epoch: 11 	Training Loss: 1.025241 	Validation Loss: 0.563729
Saving model..
Epoch: 12 	Training Loss: 1.057693 	Validation Loss: 0.535628
Saving model..
Epoch: 13 	Training Loss: 1.024526 	Validation Loss: 0.557569
Epoch: 14 	Training Loss: 1.038278 	Validation Loss: 0.565381
Epoch:

In [ ]:
resnet18.load_state_dict(torch.load('resnet18.pt'))

<All keys matched successfully>

In [ ]:
def test(loaders, model, criterion, device):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU or CPU
        data, target = data.to(device), target.to(device)
        
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [ ]:
test(dataloaders, resnet18, criterion_scratch, device)

Test Loss: 0.516238


Test Accuracy: 82% (693/836)
